## Live-Coding lesson on Markov chains

We'll apply to text generation

In [4]:
text_file = "./divinacommedia_cleaned.txt"

## Familiarize with data

In [5]:
with open(text_file, 'r', encoding='utf8') as infile:
    for line in infile:
        pass

How many words I find in my file?
How many couples I need?

I have 30 characters possible( 26 letters + space, newline, full stop, exlamation point)

In [6]:
#First order Chain
30**2

900

In [7]:
#Second order Chain
30**3

27000

In [8]:
#third order Chain
30**4

810000

I need at leat 10e7 characters to make a good estimations of that order

In our corpus we have .ca 5e5 characters: we can't do a good 2nd order estimation. But we know that not every combination is spread equally, some compbination doesn't occur at all!!! Most of the combination won't appear

In [9]:
letters=0
with open(text_file, 'r', encoding='utf8') as infile:
    for line in infile:
        for letter in line:
            letters +=1
print(letters)

529771


In [10]:
# How many and which letters I actually have in my text
from collections import Counter
observed = Counter()
with open(text_file, 'r', encoding='utf-8-sig') as infile:
    for line in infile:
        for letter in line:
            observed[letter] += 1

In [11]:
observed

Counter({'N': 258,
         'e': 46094,
         'l': 23087,
         ' ': 83019,
         'm': 11681,
         'z': 1848,
         'o': 37254,
         'd': 14599,
         'c': 20267,
         'a': 42035,
         'i': 39200,
         'n': 26229,
         's': 22188,
         't': 22478,
         'r': 25805,
         'v': 7946,
         '\n': 19053,
         'p': 10790,
         'u': 13408,
         ',': 8513,
         'h': 7109,
         'é': 903,
         '.': 3275,
         'A': 377,
         'q': 3025,
         'è': 925,
         'g': 7121,
         'f': 4947,
         '!': 232,
         'T': 283,
         '’': 7623,
         'ù': 1080,
         ';': 1628,
         'b': 2758,
         'ò': 938,
         'I': 359,
         'M': 405,
         'à': 855,
         'E': 586,
         'ì': 1383,
         'P': 473,
         'Q': 328,
         '«': 1062,
         '»': 1062,
         'R': 117,
         ':': 988,
         'ï': 427,
         'ó': 30,
         '?': 278,
         'O': 356,
   

We can treat this wierd data as they are, or we can replace them in a certain way.
We make a manual normalization

In [12]:
to_replace = {'Ë':'E', 'Ï':'I',
              'ö':'o', 'ä':'a',
              'ü':'u', 'ë':'e',
              'ï':'i' }

def letter_normalization_naive(letter):
    if letter in to_replace:
        return to_replace[letter]
    return letter
 
def letter_normalization_short(letter):
    return to_replace.get(letter, letter)    
    

In [13]:
letter_normalization_naive('Ë')

'E'

In [14]:
letter_normalization_short('Ë')

'E'

In [15]:
%timeit letter_normalization_naive('Ë')

125 ns ± 7.28 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [16]:
%timeit letter_normalization_short('Ë')

160 ns ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


It's slightly faster the naive version!

In [17]:
letter_normalization = letter_normalization_naive

In [18]:
# How many and which letters I actually have in my text
from collections import Counter
observed = Counter()
with open(text_file, 'r', encoding='utf-8-sig') as infile:
    for line in infile:
        for letter in line:
            modified_letter = letter_normalization(letter)
            observed[modified_letter] += 1

In [19]:
from random import choices

completely random selection!!!

In [20]:
letters = list(observed.keys())
occurences = list(observed.values())
generated = choices(letters, occurences, k =20 )

collated = "".join(generated)
print(collated)

acua rrlu nrall
aoht


# First order markov chain

We need to take letters in couples.

"home" -> (h,o), (o,m), (m,e)

In [21]:
def couples_from_seq(seq):
    """`seq` is a list of characters"""
    return zip(seq, seq[1:])

list(couples_from_seq('home'))

[('h', 'o'), ('o', 'm'), ('m', 'e')]

Let's pick up all the divina commedia as a single string:

### NOT MEMORY FRIENDLY!!!!

In [22]:
with open(text_file, 'r', encoding='utf-8-sig') as infile:
    whole_text = "".join(line for line in infile)

In [23]:
print(whole_text[:120])

Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura,
ché la diritta via era smarrita.

Ahi quanto a dir


In [24]:
all_couples = list(couples_from_seq(whole_text))

In [25]:
all_couples[:10]

[('N', 'e'),
 ('e', 'l'),
 ('l', ' '),
 (' ', 'm'),
 ('m', 'e'),
 ('e', 'z'),
 ('z', 'z'),
 ('z', 'o'),
 ('o', ' '),
 (' ', 'd')]

In [26]:
from collections import defaultdict
# Counter -= defaultdict(int)
#I think it's like a counter of counters
couples_counter = defaultdict(Counter)
for first_letter, second_letter in all_couples:
    couples_counter[first_letter][second_letter] +=1

In [27]:
foo = defaultdict(Counter)
foo['b']['a'] += 1
foo['b']['c'] += 1

#Is like:
if 'c' not in foo:
    foo['c'] = Counter()
foo['c']['a'] += 1

foo

foo['b']

Counter({'a': 1, 'c': 1})

How many actual couples did we observe:

In [28]:
sum(sum(counts.values()) for counts in couples_counter.values())

529769

In [29]:
letter = 'N'
possible_letters = list(couples_counter[letter].keys())
counts = list(couples_counter[letter].values())
choices(possible_letters, counts)[0]

'é'

In order to make the simulation I need to repeat this procedure replacing the starting letter

In [30]:
text = ['N']

In [31]:
for i in range(200):
    last_letter = text[-1]
    possible_letters = list(couples_counter[last_letter].keys())
    counts = list(couples_counter[last_letter].values())
    next_letter = choices(possible_letters, counts)[0]
    text.append(next_letter)
print("".join(text))

Novo quttolom’l ciontre ’osa doneriose cate?».
Tuto
sara? oma lo quansa i.

pe co toio v’,



dinne,
serisspo gntuammocerà «Ad sema;
Magi sttona

pi na
Mato,
No tr pi ra;
Qunone ifi atugu r a di se sav


## Second order Markov Chain

In [32]:
with open(text_file, 'r', encoding='utf-8-sig') as infile:
    whole_text = "".join(line for line in infile)

In [33]:
def triplets_from_seq(seq):
    """`seq` is a list of characters"""
    return zip(seq, seq[1:], seq[2:])

In [34]:
all_triplets = list(triplets_from_seq(whole_text))

In [35]:
all_triplets[:10]

[('N', 'e', 'l'),
 ('e', 'l', ' '),
 ('l', ' ', 'm'),
 (' ', 'm', 'e'),
 ('m', 'e', 'z'),
 ('e', 'z', 'z'),
 ('z', 'z', 'o'),
 ('z', 'o', ' '),
 ('o', ' ', 'd'),
 (' ', 'd', 'e')]

In [36]:
from collections import defaultdict
# Counter -= defaultdict(int)
#I think it's like a counter of counters
triplets_counter = defaultdict(Counter)
for first_letter, second_letter, third_letter in all_triplets:
    triplets_counter[(first_letter,second_letter)][third_letter] +=1

In [37]:
letters = tuple('Ne')
possible_letters = list(triplets_counter[letters].keys())
counts = list(triplets_counter[letters].values())
choices(possible_letters, counts)[0]

't'

In [38]:
text = ['N', 'e', 'l']
tuple(text[-2:])

('e', 'l')

In [39]:
text = ['N', 'e', 'l']
tuple(text[-2:])
last_letters = tuple(text[-2:])
possible_letters = list(triplets_counter[last_letters].keys())
counts = list(triplets_counter[last_letters].values())
choices(possible_letters, counts)[0]

'l'

In [40]:
text = ['N', 'e']
for i in range(200):
    last_letters = tuple(text[-2:])
    possible_letters = list(triplets_counter[last_letters].keys())
    counts = list(triplets_counter[last_letters].values())
    next_letter = choices(possible_letters, counts)[0]
    text.append(next_letter)
print("".join(text))

Nelle daredichia
«Deh, di ver lor lio, Trantavanzi.

Quemossanzia voi vol tutto
quessi mil suoi to benimmina vosì, la ché qua che ma,
d’udisal nomettaro aldant’ae questrospiartò che al perivane.

e al q


# Markovian chain of order N

It's not a masterpiece because the program create multiple copies of the text to merge them in a zip

In [73]:
from collections import Counter, defaultdict
from random import choices

text_file = "./divinacommedia_cleaned.txt"
with open(text_file, 'r', encoding='utf-8-sig') as infile:
    whole_text = "".join(line for line in infile)
    
def n_tuples_from_seq(n, seq): 
    tuple_of_seq = tuple(seq[i:] for i in range(n))
    return zip(*tuple_of_seq)

def counts_from_seq(tuple_seq):
    tuple_counter = defaultdict(Counter)
    for *previuos_letters, last_letter in tuple_seq: #Separate the last element from the rest
        tuple_counter[tuple(previuos_letters)][last_letter] +=1 #You can't use list as key for dictionaries
    return tuple_counter


    
def generate_text(lenght, seed, n):
    for i in range(lenght):
        last_letters = tuple(text[-(n-1):])
        possible_letters = list(tuples_counter[last_letters].keys())
        counts = list(tuples_counter[last_letters].values())
        next_letter = choices(possible_letters, counts)[0]
        text.append(next_letter)
    return text

lenght = 10
n=5
seed = list(whole_text[:n-1])
all_tuples = list(n_tuples_from_seq(n,whole_text))
tuples_counter = counts_from_seq(all_tuples)
text = generate_text(lenght, seed, n)
print("".join(text))

Nel fosse.

Ito è nostro


In [ ]:
import requests
url_base = ("https://raw.githubusercontent.com/UniboDIFABiophysics"+
                "/programmingCourseDIFA/master/divine_comedy/")
filename = "divinacommedia_cleaned.txt"
response = requests.get(url_base + filename)    
response.raise_for_status()    
with open(filename, 'wb') as handle:
    handle.write(response.content)

## Transformation in a Class
I would like to obtain an object that acts like that:
```python
whole_text = load_source()
generator = TextGenerator(order = 3, split='letters')
generator.learn(whole_text) #How many times I want
generator.generate(seed='Nel', lenght=400)

```

In [87]:
class NotLearnedError(Exception):
    pass

class TextGenerator:
    def __init__(self, order, split='letter'):
        self._learned = False #Did I ever called the learn function
        self.order = order
        self.split = split
        self.data = defaultdict(Counter)
        
    def _counts_from_seq(self, tuple_seq):
        for *previuos_letters, last_letter in self.data: #Separate the last element from the rest
            self.data[tuple(previuos_letters)][last_letter] +=1 #You can't use list as key for dictionaries

    def _generate_text_tokens(self, text):
        if self.split == 'letter':
            return list(text)
        elif self.split == 'word': #TODO: how to manage commas
            return text.split(' ') 
        else:
            raise ValueError("Splitting method not known")
            
        
    def learn(self, text):
        tokens = self._generate_text_tokens(text)
        all_tuples = list(n_tuples_from_seq(self.order,tokens))
        self._counts_from_seq(all_tuples)
        #We have learned something
        self._learned = True
        # fluent interface
        return self
    
    def generate(self, seed, lenght):
        """
        raises:
        -------
            `NotLearnedError` if the `self.learn` function has not been called before
        """
        # Check that we perfomed the learn function
        if not self._learned:
            raise NotLearnedError()
        output_text = generate_text(lenght = lenght,seed = seed, n=self.order)
        return output_text

In [84]:
generator = TextGenerator(order = 3, split='letters')
generator.learn(whole_text) #How many times I want
generator.generate(seed='Nel', lenght=400)

IndexError: list index out of range

The fluent interface allows me to make those kind of sequential calls:

In [ ]:
generator = TextGenerator(order = 3).learn(whole_text).generate(seed='Nel', lenght=400)

```python
text_1 = load_divina_commedia()
text_2 = load_decameron()



```